<a href="https://colab.research.google.com/github/DajeanArcila/biblioteca_pandas/blob/main/Redes_Neurais_Churn2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [2]:
# Carregando base de dados

df = pd.read_excel("churn.xlsx")

In [3]:
df.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,45,No,One year,No,Bank transfer,42.30,1840.75,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
#Excluindo variáveis não úteis

df = df.drop(columns=["customerID"])

In [5]:
df = df.dropna()

In [6]:
df.isnull().any()

,0
tenure,False
PhoneService,False
Contract,False
PaperlessBilling,False
PaymentMethod,False
MonthlyCharges,False
TotalCharges,False
Churn,False


In [7]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"],errors='coerce')

In [8]:
cat_vars = ['PhoneService',"Contract","PaperlessBilling","PaymentMethod"]
num_vars = ["tenure","MonthlyCharges","TotalCharges"]
modeling_vars = cat_vars + num_vars

In [9]:
X = df.filter(modeling_vars).copy()
y = df['Churn'].copy()
y = pd.DataFrame(y)

In [16]:
from sklearn.preprocessing import LabelEncoder

# Codificando a variável target
le = LabelEncoder()
y = le.fit_transform(y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
X = pd.get_dummies(X)

In [12]:
# Distruibuição da variáveis target
df["Churn"].value_counts()


,count
Churn,
No,5173
Yes,1869


In [13]:
# Dividindo a base em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

print(f"Shape X_train: {X_train.shape}")
print(f"Shape y_train: {y_train.shape}")
print(f"Shape X_test: {X_test.shape}")
print(f"Shape y_test: {y_test.shape}")

Shape X_train: (5633, 14)
Shape y_train: (5633, 1)
Shape X_test: (1409, 14)
Shape y_test: (1409, 1)


In [23]:
# Normalização das variáveis

from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

# Criando o pipeline para imputação, normalização e classificação
pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', MLPClassifier(hidden_layer_sizes=(100, 50, 20), activation='logistic', max_iter=300))
])

In [26]:

# Ajustando o pipeline
pipeline.fit(X_train, y_train)

# Predizendo no conjunto de teste
y_pred = pipeline.predict(X_test)

# Avaliando o modelo
print(f"Acurácia: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(f"F1 Score: {f1_score(y_test, y_pred) * 100:.2f}%")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Acurácia: 80.13%
F1 Score: 57.70%


In [27]:
#f1 score de 57 porciento

In [28]:
from joblib import dump, load

# Guardar el modelo
dump(clf, 'modelo_churn.joblib')

# Guardar el escalador
dump(scaler, 'scaler_churn.joblib')


['scaler_churn.joblib']

In [29]:
# Cargar el modelo y el escalador
clf = load('modelo_churn.joblib')
scaler = load('scaler_churn.joblib')

# Preprocesar los datos nuevos de la misma manera
# ...
X_new = ...  # Nuevos datos preprocesados

# Normalizar los datos
X_new_scaled = scaler.transform(X_new)

# Realizar predicciones
y_new_pred = clf.predict(X_new_scaled)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


TypeError: float() argument must be a string or a real number, not 'ellipsis'

In [ ]:
print(y_new_pred)


In [ ]:
results_df = pd.DataFrame({'Predicciones': y_new_pred})
print(results_df.head())


In [ ]:
#para ver la prediccion con los datos originales
results_df = pd.DataFrame(X_new)
results_df['Predicciones'] = y_new_pred
print(results_df.head())


In [ ]:
#interpretabilidad de datos para saber si sera o no churn
results_df['Predicciones'] = results_df['Predicciones'].map({0: 'No Churn', 1: 'Churn'})
print(results_df.head())



In [ ]:
results_df.to_excel('predicciones_churn.xsls', index=False)
